In [438]:
#Dependencies
import pandas as pd
import numpy as np
from pprint import pprint

In [439]:
#Import data
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

In [440]:
#Convert to dataframes
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

In [441]:
#Preview school data
school_data.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [442]:
#Preview student data
student_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [443]:
#Merge both dataframes
school_data_complete = pd.merge(school_data, student_data, how = "left", on=["school_name", "school_name"])

In [444]:
#Preview merged dataframe
school_data_complete.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [445]:
#Check for holes in data
school_data_complete.count()

School ID        39170
school_name      39170
type             39170
size             39170
budget           39170
Student ID       39170
student_name     39170
gender           39170
grade            39170
reading_score    39170
math_score       39170
dtype: int64

## District Summary
Create a high level snapshot (in table form) of the district's key metrics, including:
  * Total Schools
  * Total Students
  * Total Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [446]:
#Calculate the total number of schools
total_schools_count = school_data["school_name"].nunique()
total_schools_count

15

In [447]:
#Calculate the total number of students
total_students = student_data["Student ID"].count()
total_students

39170

In [448]:
#Calculate the total budget
budgets = school_data["budget"].nunique()
budgets

15

In [449]:
total_budget = school_data["budget"].sum()
total_budget

24649428

In [450]:
#Calculate the average math score
average_math_score = student_data["math_score"].mean()
average_math_score

78.98537145774827

In [451]:
#Calculate the average reading score
average_reading_score = student_data["reading_score"].mean()
average_reading_score

81.87784018381414

In [452]:
#Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
overall_average = (average_math_score+ average_reading_score)/2
overall_average

80.43160582078121

In [453]:
#Calculate the percentage of students with a passing math score (70 or greater)
total_passing_math = (student_data["math_score"]>=70).sum()
percentage_passing_math = (total_passing_math/total_students)*100
percentage_passing_math

74.9808526933878

In [454]:
#Calculate the percentage of students with a passing reading score (70 or greater)
total_passing_reading = (student_data["reading_score"]>=70).sum()
percentage_passing_reading = (total_passing_reading/total_students)*100
percentage_passing_reading

85.80546336482001

In [455]:
#Create a dataframe
district_summary = pd.DataFrame ({"Total Schools":[total_schools_count],
                                  "Total Students": [total_students],
                                  "Total Budget": [total_budget],
                                  "Average Math Score": [average_math_score],
                                  "Average Reading Score": [average_reading_score],
                                  "% Passing Math": [percentage_passing_math],
                                  "% Passing Reading": [percentage_passing_reading],
                                  "% Overall Passing Rate": [overall_average]})
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary
Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [472]:
#Find average math score, average reading score, % passing math, and % passing reading
percentages = student_data[["math_score", "reading_score", "school_name"]].copy()

#Bin those who failed and those who passed, assign value 0 or 100 so that you can get the average/percentages
percentages["passing_math"] = pd.cut(percentages["math_score"],[0,69,101], labels = [0, 100]).astype("int64")
percentages["passing_reading"] = pd.cut(percentages["reading_score"], [0,69,101], labels = [0,100]).astype("int64")

In [473]:
#Take the average of math_score, reading_score, passing_math, and passing_reading
percentages = percentages.groupby("school_name").mean()
percentages.head()

,math_score,reading_score,passing_math,passing_reading
school_name,,,,
Bailey High School,77.048432,81.033963,66.680064,81.933280
Cabrera High School,83.061895,83.975780,94.133477,97.039828
Figueroa High School,76.711767,81.158020,65.988471,80.739234
Ford High School,77.102592,80.746258,68.309602,79.299014
Griffin High School,83.351499,83.816757,93.392371,97.138965


In [474]:
#Calculate overall passing rate and add column to dataframe
percentages["overall_passing_rate"] = (percentages["passing_math"] + percentages["passing_reading"])/2
percentages.head()

,math_score,reading_score,passing_math,passing_reading,overall_passing_rate
school_name,,,,,
Bailey High School,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,83.351499,83.816757,93.392371,97.138965,95.265668


In [475]:
#Build another dataframe with school data to find the per student budget
school_stats = school_data.copy()
school_stats['per_student_budget'] = school_stats['budget'] / school_stats['size']

In [476]:
#Merge percentages and per student budget into one dataframe
school_stats = pd.merge(school_stats, percentages, on='school_name')
school_stats.head()

,School ID,school_name,type,size,budget,per_student_budget,math_score,reading_score,passing_math,passing_reading,overall_passing_rate
0,0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
1,1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
2,2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,94.860875
3,3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
4,4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668


In [477]:
#Rename columns
school_stats_clean = school_stats.rename(columns={"school_name": "School Name",
                                                  "type": "School Type", 
                                                  "size": "Total Students", 
                                                  "budget": "Total School Budget",
                                                  "per_student_budget": "Per Student Budget",
                                                  "math_score": "Average Math Score",
                                                  "reading_score": "Average Reading Score",
                                                  "passing_math":"% Passing Math", 
                                                  "passing_reading": "% Passing Reading",
                                                  "overall_passing_rate": "% Overall Passing Rate"})

In [478]:
#Set index to School Name and drop School ID column
school_stats_clean = school_stats_clean.set_index("School Name")
school_stats_clean = school_stats_clean.drop(["School ID"], axis = 1)
school_stats_clean

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,94.860875
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391


## Top Performing Schools (By Passing Rate)
Create a table that highlights the top 5 performing schools based on Overall Passing Rate. Include:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [479]:
top_performing_schools = school_stats_clean.sort_values(by="% Overall Passing Rate", ascending = False)
top_performing_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)
Create a table that highlights the bottom 5 performing schools based on Overall Passing Rate. Include all of the same metrics as above.

In [480]:
low_performing_schools = school_stats_clean.sort_values(by="% Overall Passing Rate", ascending = True)
low_performing_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade
Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [481]:
#Create a group organized by grade
group_by_grade = student_data.groupby(["school_name", "grade"]).mean().reset_index()

In [482]:
#Rearrange data so that each grade is a column, the school name is the index and average math score is the row data
math_by_grade = group_by_grade.pivot(index="school_name", columns="grade", values="math_score")

In [483]:
#Clean up dataframe and reorganize columns
math_by_grade = math_by_grade[["9th", "10th", "11th", "12th"]]
math_by_grade

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Scores by Grade
Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [484]:
#Rearrange data so that each grade is a column, the school name is the index and average reading score is the row data
reading_by_grade = group_by_grade.pivot(index="school_name", columns="grade", values="reading_score")

In [485]:
#Clean up dataframe and reorganize columns
reading_by_grade = reading_by_grade[["9th", "10th", "11th", "12th"]]
reading_by_grade

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending
Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [486]:
#Create new dataframe to hold spending ranges
spending_bins_df = school_stats_clean[["Per Student Budget", "Average Math Score", "Average Reading Score", 
                                       "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].copy()

In [487]:
#Isolate all Per Student Budget figures to determine bin range
spending_bins_df["Per Student Budget"].min()

578.0

In [488]:
#Isolate all Per Student Budget figures to determine bin range
spending_bins_df["Per Student Budget"].max()

655.0

In [489]:
#Create spending bins and labels
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [490]:
#Create spending bins column and add to dataframe
spending_bins_df["Spending Ranges (Per Student)"] = pd.cut(spending_bins_df["Per Student Budget"], 
                                                           spending_bins, labels = group_names)

In [491]:
#Groupby spending bins and calculuate averages for academic performance
spending_bins_df = spending_bins_df.groupby("Spending Ranges (Per Student)").mean()

In [492]:
#Drop Per Student Budget Column
spending_bins_df_clean = spending_bins_df.drop(["Per Student Budget"], axis = 1)
spending_bins_df_clean

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

Repeat the above breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).


In [493]:
#Create new dataframe to hold school size ranges
size_bins_df = school_stats_clean[["Total Students", "Average Math Score", "Average Reading Score", 
                                       "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].copy()

In [494]:
#Isolate all School Size figures to determine bin range
size_bins_df["Total Students"].min()

427

In [495]:
#Isolate all School Size figures to determine bin range
size_bins_df["Total Students"].max()

4976

In [496]:
#Create size bins and labels
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [497]:
#Create size bins column and add to dataframe
size_bins_df["School Size"] = pd.cut(size_bins_df["Total Students"], size_bins, labels = group_names)

In [498]:
#Groupby size bins and calculuate averages for academic performance
size_bins_df = size_bins_df.groupby("School Size").mean()

In [499]:
#Drop Total Students Column
size_bins_df_clean = size_bins_df.drop(["Total Students"], axis = 1)
size_bins_df_clean

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type
Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).

In [500]:
#Create new dataframe to hold school size ranges
school_type_df = school_stats_clean[["School Type", "Average Math Score", "Average Reading Score", 
                                       "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]].copy()

In [501]:
school_type_df.groupby("School Type").mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


Observable trends based on the data:
1.  Charter Schools are higher performing academically than District Schools despite the per student budget being higher for students attending District Schools.
2.  Math and Reading Averages per school do not differ significantly from grade to grade.
3.  Charter Schools have fewer students per school than District Schools, which may mean that the student to teacher ratio is lower and students thereby get more support to succeed academically.